In [2]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings("ignore")

In [3]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def rms(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    mean = (sum / (float)(n))
    root = math.sqrt(mean)
    return root
def mav(arr):
    n = len(arr)
    mav = sum((abs(np.array(arr))))/(float)(n)
    return mav
def var(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    result = (sum / (float)(n))
    return result
def wl(arr):
    n = len(arr)
    sum = 0
    for i in range(1, n):
        sum += abs(arr[i] - arr[i - 1])
    return sum
def iav(arr):
    return sum(abs(np.array(arr)))
def mean(arr):
    return np.sum(arr)/len(arr)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['restimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['restimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['restimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Electrodes = {}
        for e in range(1, 11):
            temp = {}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = EMG.iloc[startIndex:LastIndex, e - 1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Electrodes["Electrode{0}".format(e)] = temp
        Movements["Movement{0}".format(m)] = Electrodes
    return Movements
def extractSubjectAngles(name):
    ex1Path = 'Kinematics/' + name + '/' + name + '_E1_A1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    angles1 = ex1['angles']
    anglesDF1 = pd.DataFrame.from_dict(angles1)
    stimulus = ex1['restimulus']

    ex2Path = 'Kinematics/' + name + '/' + name + '_E2_A1.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    angles2 = ex2['angles']
    anglesDF2 = pd.DataFrame.from_dict(angles2)
    stimulus2 = ex2['restimulus']

    ex3Path = 'Kinematics/' + name + '/' + name + '_E3_A1.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    angles3 = ex3['angles']
    anglesDF3 = pd.DataFrame.from_dict(angles3)
    stimulus3 = ex3['restimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF1
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF3

        Angles = {}
        for a in range(1,23):
            if a == 6:
               Angles["Angle6"] = ""
               continue

            temp={}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = angle.iloc[startIndex:LastIndex, a-1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Angles["Angle{0}".format(a)] = temp
        Movements["Movement{0}".format(m)] = Angles

    return Movements


In [4]:
final_df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                                 'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                                 'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                                 'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                                 'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                                 'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                                 'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                                 'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                                 'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                                 'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                                 'Train'})
final_df_angle = pd.DataFrame(columns={})
df_angle = pd.DataFrame(columns={})
i=0
for s in range(1,2):
    subject = 'S' + str(s)
    df_angles = pd.DataFrame.from_dict(extractSubjectAngles(subject))
    for m in range(1,51):
        M = df_angles['Movement'+str(m)]
        start = copy.deepcopy(i)
        for a in range (1,23):
            if a==6:
                continue

            Angles = M['Angle' +str(a)]
            for r in range(1,7):
                rep = "R" + str(r)
                for x in range(0, len(Angles[rep]), 48):
                    df_angle.at[i, a-1] = mean(Angles[rep][x:x + 50])
                    i += 1
            if a != 22:
                i = copy.deepcopy(start)
    final_df_angle = final_df_angle.append(df_angle, ignore_index=True)            

Kinematics/S1/S1_E1_A1.mat


In [5]:
for s in range(1,2):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                           'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                           'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                           'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                           'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                           'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                           'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                           'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                           'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                           'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                           'Train'})
    for e in range(1, 11):
        electrode = 'Electrode' + str(e)
        i=0
        for m in range(1,51):
            M = dff['Movement'+str(m)][electrode]
            for r in range(1, 7):
                rep = "R" + str(r)
                if (r in [1, 3, 4, 6]):
                    train = 1
                else:
                    train = 0
                for x in range(0, len(M[rep]), 48):
                    df.at[i, 'RMS' + str(e)] = rms(M[rep][x:x + 50])
                    df.at[i, 'MAV' + str(e)] = mav(M[rep][x:x + 50])
                    df.at[i, 'VAR' + str(e)] = var(M[rep][x:x + 50])
                    df.at[i, 'WL' + str(e)] = wl(M[rep][x:x + 50])
                    df.at[i, 'IAV' + str(e)] = iav(M[rep][x:x + 50])
                    df.at[i, 'Train'] = train
                    i += 1
    final_df = final_df.append(df, ignore_index=True)

DB1/S1/S1_A1_E1.mat


In [6]:
final_df['CMC1_f'] = df_angle[0]
final_df['CMC1_a'] =  df_angle[1]
final_df['MCP1'] =  df_angle[2]
final_df['IP1'] =  df_angle[3]
final_df['MCP2_f'] =  df_angle[4]
final_df['PIP2'] =  df_angle[6]
final_df['MCP3_f'] =  df_angle[7]
final_df['PIP3'] =  df_angle[8]
final_df['MCP4_f'] =  df_angle[9]
final_df['MCP4_a'] =  df_angle[10]
final_df['PIP4'] =  df_angle[11]
final_df['CMC5'] =  df_angle[12]
final_df['MCP5_f'] =  df_angle[13]
final_df['MCP5_a'] =  df_angle[14]
final_df['PIP5'] =  df_angle[15]
final_df['DIP2'] =  df_angle[16]
final_df['DIP3'] =  df_angle[17]
final_df['DIP4'] =  df_angle[18]
final_df['DIP5'] =  df_angle[19]
# final_df['WRIST_F'] =  df_angle[20]
# final_df['WRIST_A'] =  df_angle[21]

features = {'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
            'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
            'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
            'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
            'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
            'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
            'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
            'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
            'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
            'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10', }

angles = {'CMC1_f','CMC1_a','MCP1','IP1','MCP2_f',
        'PIP2','MCP3_f','MCP4_f','MCP4_a',
        'PIP4','CMC5','MCP5_f','MCP5_a','PIP5',
        'DIP2','DIP4','DIP5', 'DIP3', 'PIP3'}

print(final_df)

          RMS2     IAV2      MAV6      VAR2      RMS9     WL6     RMS10  \
0     0.078694   3.0761    0.0024  0.006193  0.060138     0.0  0.008277   
1     0.091641   4.1844  0.002748  0.008398  0.068177  0.0148  0.012856   
2     0.014363   0.4308  0.002794  0.000206  0.033427  0.0098  0.006268   
3       0.0024     0.12    0.0024  0.000006  0.019749     0.0  0.003647   
4       0.0024     0.12    0.0024  0.000006  0.042136     0.0  0.006343   
...        ...      ...       ...       ...       ...     ...       ...   
2330   0.24126  11.9216  0.028758  0.058206  0.146366  0.1856  0.005907   
2331  0.201787   9.8631  0.033886  0.040718  0.151758  0.1978  0.004559   
2332  0.150342   7.2023  0.019374  0.022603   0.14272  0.1367  0.004705   
2333  0.143653   5.1438  0.045304  0.020636  0.137048  0.2663  0.030759   
2334  0.003141   0.0602    0.0577   0.00001  0.155534  0.1174  0.156204   

         IAV8      VAR5     WL5  ...     MCP4_a       PIP4      CMC5  \
0      2.6584  0.000006   0

In [7]:
X_train = final_df[final_df['Train'] == 1].loc[:, features]
scalar =  StandardScaler()
scalar = scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test = final_df[final_df['Train'] == 0].loc[:, features]
X_test = scalar.transform(X_test)

y_train = final_df[final_df['Train'] == 1].loc[:, angles]
y_test = final_df[final_df['Train'] == 0].loc[:, angles]
y_train.to_csv('angles_y_train.csv')
y_test.to_csv('angles_y_test.csv')

In [8]:
input = Input(shape =(50,))
L1 = Dense(500, activation='tanh')(input)
L2 = Dense(500, activation='sigmoid')(L1)
ouput = Dense(19, activation='linear')(L2)
model = Model(input, ouput)
model.compile(optimizer=Adam(learning_rate=0.0045), loss="mean_squared_error", metrics=['mae', 'mse'])
model.fit(X_train, y_train, epochs=1500)
print(model.evaluate(X_test, y_test))
predictions = model.predict(X_test)



Epoch 1/1500


2022-05-27 22:03:51.849273: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


48/48 [==============================] - 0s 2ms/step - loss: 880.5810 - mae: 17.7835 - mse: 880.5810
Epoch 2/1500
48/48 [==============================] - 0s 3ms/step - loss: 632.5818 - mae: 15.8834 - mse: 632.5818
Epoch 3/1500
48/48 [==============================] - 0s 4ms/step - loss: 590.4544 - mae: 15.5062 - mse: 590.4544
Epoch 4/1500
48/48 [==============================] - 0s 2ms/step - loss: 555.8066 - mae: 15.0602 - mse: 555.8066
Epoch 5/1500
48/48 [==============================] - 0s 2ms/step - loss: 530.9922 - mae: 14.7201 - mse: 530.9922
Epoch 6/1500
48/48 [==============================] - 0s 2ms/step - loss: 499.2138 - mae: 14.3025 - mse: 499.2138
Epoch 7/1500
48/48 [==============================] - 0s 2ms/step - loss: 473.2544 - mae: 13.9336 - mse: 473.2544
Epoch 8/1500
48/48 [==============================] - 0s 3ms/step - loss: 446.8239 - mae: 13.6328 - mse: 446.8239
Epoch 9/1500
48/48 [==============================] - 0s 3ms/step - loss: 421.7952 - mae: 13.2134 - m

In [69]:
predictions = model.predict(X_test)
length = predictions.shape[0]
# print(predictions[1])
y_test.reset_index(drop=True, inplace=True)
#print(y_test.loc[0,:])

In [1]:
Error = pd.DataFrame(columns =['PIP4','DIP5','PIP3','MCP5_a','DIP4','MCP2_f','DIP2','MCP3_f','IP1','PIP5','PIP2','MCP1','CMC1_a','MCP5_f','MCP4_a','MCP4_f','CMC1_f','DIP3','CMC5'])
list = Error.columns.values.tolist()
print(list)
for i in range(0, length):
    ActualI = y_test.loc[i,:]
    predictedI = predictions[i]
    print(predictedI)
    k = 0
    for j in list:
        print(predictedI[k])
        difference =  abs(predictedI[k] - ActualI[k])
        print(j)
        Error.at[i, j] = difference
        k+=1
        print(Error)


print(Error)        






NameError: name 'pd' is not defined

In [11]:
-12.842432671465666,13.418102414241673,4.101825888519285,-18.842589280331165,-25.604314583244467,11.695412868578613,40.187775547894965,26.402826169380774,19.857298752862327,15.766037398552712,90.43685610817194,1.1888788558435552,9.815728089925392,54.61345891955365,0.5178820980259151,10.081088039618288,0.7238138389000116,14.175547125216255,1.9287114778764098
7,-13.400147981938181,12.482409253232285,2.8571009475906872,-16.778192450249893,-14.271617305527272,16.37884656727018,50.32203490701934,23.231383560336504,12.451081320288129,15.290323288525048,97.88816358590137,-0.6498964609565379,8.073109697141408,54.066150784239845,-0.0844717931259007,10.272925129176242,5.965760648362786,23.288225177550547,1.9287114778764098
27.225043452500504,28.295868195239255,-11.454231605843008,-0.391457008278672,4.253836695248956,-65.89292141911842,9.611844583887207,-6.593315193065799,-38.21023473140616,11.827818390737072,25.854688563497184,6.920610113342703,2.1119786139799888,1.0333574325250479,14.897568576810711,10.73810029759642,9.395141182097142,-0.19959425116956603,-12.523755980490446

(27.225043452500504,
 28.295868195239255,
 -11.454231605843008,
 -0.391457008278672,
 4.253836695248956,
 -65.89292141911842,
 9.611844583887207,
 -6.593315193065799,
 -38.21023473140616,
 11.827818390737072,
 25.854688563497184,
 6.920610113342703,
 2.1119786139799888,
 1.0333574325250479,
 14.897568576810711,
 10.73810029759642,
 9.395141182097142,
 -0.19959425116956603,
 -12.523755980490446)